In [1]:
from Acesso import Login
from Query import Query
from Moeda import Moeda
from RemoverArquivo import Remover
from Email import Email
from glob import glob
import os
import pandas as pd

s=Login()

sql=Query(s.usuario,s.senha,s.database,s.server)

querys={

    'Vendas':

    """
    
    DECLARE @DTBASE DATETIME, @DTFIM DATETIME, @DTINICIO DATETIME

    SET @DTBASE=CONVERT(DATETIME,CAST(GETDATE() AS DATE),101)

    SET @DTFIM=@DTBASE

    SET @DTINICIO=DATEADD(DAY,(DAY(@DTFIM)-1)*-1,@DTFIM)

    SELECT b.[ID Vendedor],s.[Email Sup],
    b.Fabricante,SUM(b.[Total Venda]) AS [Total Venda] 
    FROM (

        SELECT a.[ID Vendedor],
        CASE WHEN p.Fabricante='DE MARCHI' THEN 'DE MARCHI' ELSE 'OUTROS' END AS Fabricante,
        a.[Total Venda]
        FROM netfeira.vw_venda_estatico a
        INNER JOIN netfeira.vw_produto p ON a.SKU=p.SKU
        WHERE [Data de Faturamento] BETWEEN @DTINICIO AND @DTFIM AND [Tipo de Operação]='VENDAS'

    )b
    INNER JOIN netfeira.vw_vendedor v ON b.[ID Vendedor]=v.[ID Vendedor]
    INNER JOIN netfeira.vw_supervisor s ON v.[ID Equipe]=s.[ID Equipe]
    GROUP BY b.[ID Vendedor],b.Fabricante,s.[Email Sup]
        
    """,

    'Calendario':

    """
    
    DECLARE @DTBASE DATETIME

    SET @DTBASE=CONVERT(DATETIME,CAST(GETDATE() AS DATE),101)

    SELECT *
    FROM netfeira.vw_calend
    WHERE YEAR(Data)=YEAR(@DTBASE) AND MONTH(Data)=MONTH(@DTBASE)
    AND [Dia Útil]=1
    
    """


}

In [2]:
df=sql.CriarTabela(kwargs=querys)

In [3]:
dia_util=df['Calendario']['Data'].count()

dia_trab=df['Calendario'].loc[df['Calendario']['Data Trabalhada'].notnull(),'Data'].count()-1

dia_trab

5

In [4]:
arquivos=glob('Metas\*.xlsx')

excel_df=pd.read_excel(arquivos[-1])

colunas=[l for l in excel_df.columns if str(l).find('Meta')>=0]

for c in colunas:
    
    excel_df[c]=excel_df[c].astype(float)
    
    excel_df[f'{c} Diária']=round(excel_df[c]/dia_util,2)
    
    pass

df['Consolidado']=df['Vendas'].pivot(index=['ID Vendedor','Email Sup'],columns='Fabricante',values='Total Venda').reset_index()

for c in ['DE MARCHI','OUTROS']:

    df['Consolidado'].loc[df['Consolidado'][c].isnull(),c]=0
        
    pass

df['Consolidado']['Total Geral']=df['Consolidado']['DE MARCHI']+df['Consolidado']['OUTROS']

for c in ['DE MARCHI','OUTROS','Total Geral']:

    df['Consolidado'][f'Projeção {c}']=round(((df['Consolidado'][c]/dia_trab)*dia_util),2)
        
    pass

excel_df=excel_df.merge(df['Consolidado'],on='ID Vendedor',how='inner')

excel_df['DE MARCHI %']=excel_df.apply(lambda info: round(info['DE MARCHI']/info['Meta DeMarchi'],4)*100 if info['Meta DeMarchi']>0 else 0,axis=1)

excel_df['OUTROS %']=excel_df.apply(lambda info: round(info['OUTROS']/info['Meta Outros'],4)*100 if info['Meta Outros']>0 else 0,axis=1)

excel_df['Meta %']=excel_df.apply(lambda info: round(info['Total Geral']/info['Meta R$'],4)*100 if info['Meta R$']>0 else 0,axis=1)

excel_df[['ID Vendedor', 'Vendedor', 'Equipe', 'Supervisor','Email Sup', 'Meta DeMarchi',
       'Meta Outros', 'Meta R$', 'Meta DeMarchi Diária', 'Meta Outros Diária',
       'Meta R$ Diária', 'DE MARCHI', 'OUTROS', 'Total Geral',
       'Projeção DE MARCHI', 'Projeção OUTROS', 'Projeção Total Geral',
       'DE MARCHI %', 'OUTROS %', 'Meta %']]

,ID Vendedor,Vendedor,Equipe,Supervisor,Email Sup,Meta DeMarchi,Meta Outros,Meta R$,Meta DeMarchi Diária,Meta Outros Diária,Meta R$ Diária,DE MARCHI,OUTROS,Total Geral,Projeção DE MARCHI,Projeção OUTROS,Projeção Total Geral,DE MARCHI %,OUTROS %,Meta %
0,ADEMIRFE,ADEMIR FERNANDES KLEIN,EQUIPE 2 (REP),ULISSES BACCI,ULISSES@DEMARCHIBRASIL.COM.BR,36075.0,19425.00,55500.00,1568.48,844.57,2413.04,10226.74,4462.11,14688.85,47043.00,20525.71,67568.71,28.35,22.97,26.47
1,ALEJESUS,ALEXANDRE DE JESUS OLIVEI REPRESENTACOES,EQUIPE 4 (VAR),DOUGLAS TEDESCHI,DOUGLASMARTINS@DEMARCHIBRASIL.COM.BR,0.0,274.13,274.13,0.00,11.92,11.92,10287.64,0.00,10287.64,47323.14,0.00,47323.14,0.00,0.00,3752.83
2,ALENOGUE,ALEXANDRE NOGUEIRA ANTONOF,EQUIPE 4 (VAR),DOUGLAS TEDESCHI,DOUGLASMARTINS@DEMARCHIBRASIL.COM.BR,0.0,20000.00,20000.00,0.00,869.57,869.57,0.00,302.93,302.93,0.00,1393.48,1393.48,0.00,1.51,1.51
3,CELSOCAR,CELSO CARDOSO AMARO,EQUIPE 1 (REP),ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,45180.0,80320.00,125500.00,1964.35,3492.17,5456.52,13022.49,20597.21,33619.70,59903.45,94747.17,154650.62,28.82,25.64,26.79
4,FONTOLAN,CELSO FONTOLAN MOLINA,EQUIPE 3 (CLT),ULISSES BACCI,ULISSES@DEMARCHIBRASIL.COM.BR,8750.0,26250.00,35000.00,380.43,1141.30,1521.74,225.68,2787.88,3013.56,1038.13,12824.25,13862.38,2.58,10.62,8.61
5,CRISTIAN,CHRISTIAN CÉSAR OLIVEIRA,EQUIPE 6 (CLT),CARLOS PASSOS,CARLOS.HENRIQUE@DEMARCHIBRASIL.COM.BR,26640.0,45360.00,72000.00,1158.26,1972.17,3130.43,3193.98,23161.03,26355.01,14692.31,106540.74,121233.05,11.99,51.06,36.60
6,CLAUDIOP,CLAUDIO TADEU PEREIRA,EQUIPE 3 (CLT),ULISSES BACCI,ULISSES@DEMARCHIBRASIL.COM.BR,52200.0,78300.00,130500.00,2269.57,3404.35,5673.91,11999.22,31937.18,43936.40,55196.41,146911.03,202107.44,22.99,40.79,33.67
7,DOUGLASM,DOUGLAS MARTINS TEDESCHI,EQUIPE 4 (VAR),DOUGLAS TEDESCHI,DOUGLASMARTINS@DEMARCHIBRASIL.COM.BR,0.0,135500.00,135500.00,0.00,5891.30,5891.30,28847.93,30065.54,58913.47,132700.48,138301.48,271001.96,0.00,22.19,43.48
8,ELENICEV,ELENICE VIRGINIA DAL POGGETTO FLEMING,EQUIPE 1 (REP),ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,15525.0,18975.00,34500.00,675.00,825.00,1500.00,2721.54,4732.56,7454.10,12519.08,21769.78,34288.86,17.53,24.94,21.61
9,FABIOARA,FABIO ARAUJO,EQUIPE 1 (REP),ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,18125.0,54375.00,72500.00,788.04,2364.13,3152.17,1895.68,15753.77,17649.45,8720.13,72467.34,81187.47,10.46,28.97,24.34
